# IBM Applied Data Science Capstone Course by Coursera
### Week 5 Final Report
**_Opening a New Gym in Mumbai, India_**
- Build a dataframe of neighborhoods in Mumbai, India by web scraping the data from Wikipedia page
- Get the geographical coordinates of the neighborhoods
- Obtain the venue data for the neighborhoods from Foursquare API
- Explore and cluster the neighborhoods
- Select the best cluster to open a new Gym
***
### 1. Import libraries

In [19]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


### 2. Scrap data from Wikipedia page into a DataFrame

In [20]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Mumbai')[0]

In [21]:
df.head()

,Area,Location,Latitude,Longitude
0,Amboli,"Andheri,Western Suburbs",19.129300,72.843400
1,"Chakala\tAndheri,",Western Suburbs,19.111388,72.860833
2,D.N. Nagar,"Andheri,Western Suburbs",19.124085,72.831373
3,Four Bungalows,"Andheri,Western Suburbs",19.124714,72.827210
4,Lokhandwala,"Andheri,Western Suburbs",19.130815,72.829270


In [22]:
# print the number of rows of the dataframe
df.shape

(93, 4)

In [23]:
# save the DataFrame as CSV file
df.to_csv("df.csv", index=False)

### 3. Create a map of Mumbai with neighborhoods superimposed on top

In [24]:
# get the coordinates of Kuala Lumpur
address = 'Mumbai , India'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Mumbai, India {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Mumbai, India 18.9387711, 72.8353355.


In [25]:
# create map of Toronto using latitude and longitude values
map_m = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(df['Latitude'], df['Longitude'], df['Area']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_m)  
    
map_m

In [26]:
# save the map as HTML file
map_m.save('map_m.html')

### 4. Use the Foursquare API to explore the neighborhoods

In [27]:
# define Foursquare Credentials and Version
CLIENT_ID = 'YSOX2ECJYC4AWP4FU5BQBS1JIYB05EFHALS1EBPR505NYXXQ' # your Foursquare ID
CLIENT_SECRET = 'XI20NAL413MC1WXRFN5KQH0NDJO3R4LUPLLEJTWL0EZG23LD' # your Foursquare Secret
VERSION = '20200616' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: YSOX2ECJYC4AWP4FU5BQBS1JIYB05EFHALS1EBPR505NYXXQ
CLIENT_SECRET:XI20NAL413MC1WXRFN5KQH0NDJO3R4LUPLLEJTWL0EZG23LD


**Now, let's get the top 100 venues that are within a radius of 2000 meters.**

In [28]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(df['Latitude'], df['Longitude'], df['Area']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [29]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Area', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(7044, 7)


,Area,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Amboli,19.1293,72.8434,Shawarma Factory,19.124591,72.840398,Falafel Restaurant
1,Amboli,19.1293,72.8434,Merwans Cake shop,19.119300,72.845418,Bakery
2,Amboli,19.1293,72.8434,Jaffer Bhai's Delhi Darbar,19.137714,72.845909,Mughlai Restaurant
3,Amboli,19.1293,72.8434,Hard Rock Cafe Andheri,19.135995,72.835335,American Restaurant
4,Amboli,19.1293,72.8434,Pizza Express,19.131893,72.834668,Pizza Place


**Let's check how many venues were returned for each neighorhood**

In [30]:
venues_df.groupby(["Area"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Area,,,,,,
Aarey Milk Colony,19,19,19,19,19,19
Agripada,100,100,100,100,100,100
Altamount Road,100,100,100,100,100,100
Amboli,89,89,89,89,89,89
Amrut Nagar,100,100,100,100,100,100
Asalfa,100,100,100,100,100,100
Ballard Estate,91,91,91,91,91,91
Bandstand Promenade,76,76,76,76,76,76
Bangur Nagar,100,100,100,100,100,100


**Let's find out how many unique categories can be curated from all the returned venues**

In [31]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 224 uniques categories.


In [32]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Falafel Restaurant', 'Bakery', 'Mughlai Restaurant',
       'American Restaurant', 'Pizza Place', 'Sandwich Place', 'Brewery',
       'Chinese Restaurant', 'Pub', 'Multiplex', 'Ice Cream Shop',
       'Indian Restaurant', 'Café', 'Mediterranean Restaurant',
       'BBQ Joint', 'Lounge', 'Diner', 'Coffee Shop', 'Juice Bar', 'Spa',
       'Jewelry Store', 'Fast Food Restaurant', 'Comfort Food Restaurant',
       'Bar', 'Gym', 'Dessert Shop', 'Italian Restaurant',
       'Movie Theater', "Women's Store",
       'Residential Building (Apartment / Condo)', 'Deli / Bodega',
       'Snack Place', 'Asian Restaurant', 'Seafood Restaurant',
       'Department Store', 'Vegetarian / Vegan Restaurant',
       'Clothing Store', 'Fish Market', 'Gym / Fitness Center', 'Hotel',
       'Camera Store', 'Shopping Mall', 'Market', 'Pharmacy',
       'Restaurant', 'Cocktail Bar', 'Maharashtrian Restaurant',
       'Food Truck', 'Airport Service', 'Nightclub'], dtype=object)

In [34]:
# check if the results contain "Shopping Mall"
"Neighborhood" in venues_df['VenueCategory'].unique()

True

### 5. Analyze Each Neighborhood

In [35]:
# one hot encoding
m_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
m_onehot['Areas'] = venues_df['Area']

# move neighborhood column to the first column
fixed_columns = [m_onehot.columns[-1]] + list(m_onehot.columns[:-1])
m_onehot = m_onehot[fixed_columns]

print(m_onehot.shape)
m_onehot.head()

(7044, 225)


,Areas,Accessories Store,Afghan Restaurant,Airport,Airport Lounge,Airport Service,American Restaurant,Antique Shop,Arcade,Art Gallery,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Beach,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Cafeteria,Café,Camera Store,Chaat Place,Cheese Shop,Chinese Restaurant,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Auditorium,College Gym,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Dive Bar,Donut Shop,Dumpling Restaurant,Electronics Store,English Restaurant,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,General College & University,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Goan Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hawaiian Restaurant,Historic Site,History Museum,Hockey Arena,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Lake,Light Rail Station,Lighthouse,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,North Indian Restaurant,Office,Opera House,Optical Shop,Other Great Outdoors,Outdoors & Recreation,Paper / Office Supplies Store,Park,Parsi Restaurant,Performing Arts Venue,Pharmacy,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Pub,Public Art,Punjabi Restaurant,Racetrack,Recreation Center,Residential Building (Apartment / Condo),Resort,Restaurant,Road,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Ski Chalet,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,South Indian Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Toy / Game Store,Track,Track Stadium,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Store,Water Park,Whisky Bar,Wine Bar,Women's Store,Yoga Studio,Zoo
0,Amboli,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Amboli,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

**Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category**

In [36]:
m_grouped = m_onehot.groupby(["Areas"]).mean().reset_index()

print(m_grouped.shape)
m_grouped

(92, 225)


,Areas,Accessories Store,Afghan Restaurant,Airport,Airport Lounge,Airport Service,American Restaurant,Antique Shop,Arcade,Art Gallery,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Beach,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Cafeteria,Café,Camera Store,Chaat Place,Cheese Shop,Chinese Restaurant,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Auditorium,College Gym,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Dive Bar,Donut Shop,Dumpling Restaurant,Electronics Store,English Restaurant,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,General College & University,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Goan Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hawaiian Restaurant,Historic Site,History Museum,Hockey Arena,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Lake,Light Rail Station,Lighthouse,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,North Indian Restaurant,Office,Opera House,Optical Shop,Other Great Outdoors,Outdoors & Recreation,Paper / Office Supplies Store,Park,Parsi Restaurant,Performing Arts Venue,Pharmacy,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Pub,Public Art,Punjabi Restaurant,Racetrack,Recreation Center,Residential Building (Apartment / Condo),Resort,Restaurant,Road,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Ski Chalet,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,South Indian Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Toy / Game Store,Track,Track Stadium,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Store,Water Park,Whisky Bar,Wine Bar,Women's Store,Yoga Studio,Zoo
0,Aarey Milk Colony,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.052632,0.000000,0.00,0.000000,0.052632,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.052632,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.052632,0.00,0.000000,0.052632,0.00,0.052632,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,

In [37]:
len(m_grouped[m_grouped["Gym"] > 0])

37

**Create a new DataFrame for Gym data only**

In [39]:
m_gym = m_grouped[["Areas","Gym"]]

In [41]:
m_gym.head()

,Areas,Gym
0,Aarey Milk Colony,0.052632
1,Agripada,0.000000
2,Altamount Road,0.010000
3,Amboli,0.022472
4,Amrut Nagar,0.010000


### 6. Cluster Neighborhoods
Run k-means to cluster the neighborhoods in Mumbai into 3 clusters.

In [44]:
# set number of clusters
kclusters = 3

m_clustering = m_gym.drop(["Areas"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(m_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 0, 2, 2, 2, 0, 0, 2, 2, 0])

In [45]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
m_merged = m_gym.copy()

# add clustering labels
m_merged["Cluster Labels"] = kmeans.labels_

In [46]:
m_merged.rename(columns={"Areas": "Area"}, inplace=True)
m_merged.head()

,Area,Gym,Cluster Labels
0,Aarey Milk Colony,0.052632,1
1,Agripada,0.000000,0
2,Altamount Road,0.010000,2
3,Amboli,0.022472,2
4,Amrut Nagar,0.010000,2


In [47]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
m_merged = m_merged.join(df.set_index("Area"), on="Area")

print(m_merged.shape)
m_merged.head() # check the last columns!

(92, 6)


,Area,Gym,Cluster Labels,Location,Latitude,Longitude
0,Aarey Milk Colony,0.052632,1,"Goregaon,Western Suburbs",19.148493,72.881756
1,Agripada,0.000000,0,South Mumbai,18.977700,72.827300
2,Altamount Road,0.010000,2,South Mumbai,18.968100,72.809500
3,Amboli,0.022472,2,"Andheri,Western Suburbs",19.129300,72.843400
4,Amrut Nagar,0.010000,2,"Ghatkopar,Eastern Suburbs",19.102077,72.912835


In [48]:
# sort the results by Cluster Labels
print(m_merged.shape)
m_merged.sort_values(["Cluster Labels"], inplace=True)
m_merged

(92, 6)


,Area,Gym,Cluster Labels,Location,Latitude,Longitude
91,Worli,0.000000,0,South Mumbai,19.000000,72.815000
71,Nehru Nagar,0.000000,0,"Kurla,Eastern Suburbs",15.451686,74.971977
30,Dava Bazaar,0.000000,0,South Mumbai,18.946882,72.831362
69,Nariman Point,0.000000,0,South Mumbai,18.926000,72.823000
33,Dhobitalao,0.000000,0,South Mumbai,18.943300,72.828600
68,Nalasopara,0.000000,0,"Vasai,western Suburbs",19.415400,72.861300
35,Dongri,0.000000,0,South Mumbai,19.283333,72.783333
36,Fanas Wadi,0.000000,0,"Kalbadevi,South Mumbai",18.951811,72.825309
67,Naigaon,0.000000,0,"Vasai,western Suburbs",19.351467,72.846343
38,Gorai,0.000000,0,"Borivali (West),Western Suburbs",19.250057,72.782021


**Finally, let's visualize the resulting clusters**

In [49]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(m_merged['Latitude'], m_merged['Longitude'], m_merged['Area'], m_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [50]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

### 7. Examine Clusters

#### Cluster 0

In [51]:
m_merged.loc[m_merged['Cluster Labels'] == 0]

,Area,Gym,Cluster Labels,Location,Latitude,Longitude
91,Worli,0.0,0,South Mumbai,19.000000,72.815000
71,Nehru Nagar,0.0,0,"Kurla,Eastern Suburbs",15.451686,74.971977
30,Dava Bazaar,0.0,0,South Mumbai,18.946882,72.831362
69,Nariman Point,0.0,0,South Mumbai,18.926000,72.823000
33,Dhobitalao,0.0,0,South Mumbai,18.943300,72.828600
68,Nalasopara,0.0,0,"Vasai,western Suburbs",19.415400,72.861300
35,Dongri,0.0,0,South Mumbai,19.283333,72.783333
36,Fanas Wadi,0.0,0,"Kalbadevi,South Mumbai",18.951811,72.825309
67,Naigaon,0.0,0,"Vasai,western Suburbs",19.351467,72.846343
38,Gorai,0.0,0,"Borivali (West),Western Suburbs",19.250057,72.782021


#### Cluster 1

In [52]:
m_merged.loc[m_merged['Cluster Labels'] == 1]

,Area,Gym,Cluster Labels,Location,Latitude,Longitude
66,Nahur,0.043478,1,"Mulund,Eastern Suburbs",19.157000,72.941000
70,Navy Nagar,0.050000,1,"Colaba,South Mumbai",18.901200,72.810100
59,Mankhurd,0.076923,1,Harbour Suburbs,19.050000,72.930000
0,Aarey Milk Colony,0.052632,1,"Goregaon,Western Suburbs",19.148493,72.881756
24,Cuffe Parade,0.035714,1,South Mumbai,18.910000,72.810000
41,I.C. Colony,0.063291,1,"Borivali (West),Western Suburbs",19.247039,72.849830


#### Cluster 2

In [53]:
m_merged.loc[m_merged['Cluster Labels'] == 2]

,Area,Gym,Cluster Labels,Location,Latitude,Longitude
2,Altamount Road,0.010000,2,South Mumbai,18.968100,72.809500
88,Vile Parle,0.010000,2,western Suburbs,19.100000,72.830000
3,Amboli,0.022472,2,"Andheri,Western Suburbs",19.129300,72.843400
4,Amrut Nagar,0.010000,2,"Ghatkopar,Eastern Suburbs",19.102077,72.912835
7,Bandstand Promenade,0.013158,2,"Bandra,Western Suburbs",19.042718,72.819132
8,Bangur Nagar,0.030000,2,"Goregaon,Western Suburbs",19.167362,72.832252
13,C.G.S. colony,0.010000,2,"Antop Hill,South Mumbai",19.016378,72.856629
82,Thakur village,0.015873,2,"Kandivali east,Western Suburbs",19.210206,72.872980
81,Sunder Nagar,0.030000,2,"Malad,western Suburbs",19.175000,72.842000
80,Sion,0.010309,2,South Mumbai,19.040000,72.860000


#### Observations:
Most of the Gyms are concentrated in the central area of Mumbai city, with the highest number in cluster 0 and moderate number in cluster 2. On the other hand, cluster 1 has very low number to totally no Gyms in the neighborhoods. This represents a great opportunity and high potential areas to open new Gyms as there is very little to no competition from existing Gyms. Meanwhile, Gyms in cluster 0 are likely suffering from intense competition due to oversupply and high concentration of Gyms. From another perspective, this also shows that the oversupply of Gyms mostly happened in the central area of the city, with the suburb area still have very few Gyms. Therefore, this project recommends property developers to capitalize on these findings to open new Gyms in neighborhoods in cluster 1 with little to no competition. Property developers with unique selling propositions to stand out from the competition can also open new Gyms in neighborhoods in cluster 2 with moderate competition. Lastly, property developers are advised to avoid neighborhoods in cluster 0 which already have high concentration of Gyms and suffering from intense competition.